In [2]:
%%time
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , precision_recall_curve , f1_score
from sklearn.metrics import classification_report , roc_auc_score, recall_score
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from joblib import dump, load
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
#Import Models
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
print("Numpy version: {}".format(np.__version__))
print("Pandas version: {}".format(pd.__version__))
import sklearn as sklearn
print("Sklearn version: {}".format(sklearn.__version__))
import keras as keras
print("Keras version: {}".format(keras.__version__))
#print("XBG Boost version: {}".format(xgboost.__version__))
import matplotlib as mpl
print ("Matplotlib version: {}".format(mpl.__version__))

Using TensorFlow backend.


Numpy version: 1.17.2
Pandas version: 0.25.1
Sklearn version: 0.21.3
Keras version: 2.2.4
Matplotlib version: 3.1.2
CPU times: user 3.88 s, sys: 1.35 s, total: 5.23 s
Wall time: 8.98 s


In [3]:
%%time
# Read csv file into dataframe
df = pd.read_csv('paysim_features.csv').drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 20 columns):
step              int64
amount            float64
ChangeOrig        float64
ChangeDest        float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
type_CASH_IN      int64
type_CASH_OUT     int64
type_DEBIT        int64
type_PAYMENT      int64
type_TRANSFER     int64
steptime          object
dayofweek         int64
timeofday         float64
dtypes: float64(8), int64(9), object(3)
memory usage: 970.9+ MB
CPU times: user 20.1 s, sys: 4.71 s, total: 24.8 s
Wall time: 26.1 s


In [4]:
df.describe()

,step,amount,ChangeOrig,ChangeDest,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,dayofweek,timeofday
count,"6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00"
mean,243.40,"179,861.90","21,230.56","124,294.73","833,883.10","855,113.67","1,100,701.67","1,224,996.40",0.00,0.00,0.22,0.35,0.01,0.34,0.08,3.12,15.32
std,142.33,"603,858.23","146,643.29","812,939.08","2,888,242.67","2,924,048.50","3,399,180.11","3,674,128.94",0.04,0.00,0.41,0.48,0.08,0.47,0.28,1.65,4.32
min,1.00,0.00,"-10,000,000.00","-13,060,826.21",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,156.00,"13,389.57","-10,150.44",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,12.00
50%,239.00,"74,871.94",0.00,0.00,"14,208.00",0.00,"132,705.66","214,661.44",0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,16.00
75%,335.00,"208,721.48",0.00,"149,105.43","107,315.18","144,258.41","943,036.71","1,111,909.25",0.00,0.00,0.00,1.00,0.00,1.00,0.00,4.00,19.00
max,743.00,"92,445,516.64","1,915,267.90","105,687,838.82","59,585,040.37","49,585,040.37","356,015,889.35","356,179,278.92",1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.00,23.00


In [5]:
df.head()

,step,amount,ChangeOrig,ChangeDest,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,steptime,dayofweek,timeofday
0,1,"9,839.64","-9,839.64",0.00,C1231006815,"170,136.00","160,296.36",M1979787155,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00
1,1,"1,864.28","-1,864.28",0.00,C1666544295,"21,249.00","19,384.72",M2044282225,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00
2,1,181.00,-181.00,0.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,0,0,0,0,1,1970-01-01 01:00:00,3,1.00
3,1,181.00,-181.00,"-21,182.00",C840083671,181.00,0.00,C38997010,"21,182.00",0.00,1,0,0,1,0,0,0,1970-01-01 01:00:00,3,1.00
4,1,"11,668.14","-11,668.14",0.00,C2048537720,"41,554.00","29,885.86",M1230701703,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00


In [6]:
# Define X with selected columns, y as the binary variable to predict
X = df.drop(['isFraud' , 'isFlaggedFraud' , 'nameOrig' , 'nameDest' ,'steptime'] , axis = 1 ).values
y = df['isFraud'].values
# Train, Test, Split to segregate Training from Test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
X.shape

(6362620, 15)

In [7]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('lr', LogisticRegression(solver='lbfgs'))
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'lr__C': [1],
              'lr__max_iter': [100],
              'lr__solver': ['lbfgs']
             }

# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

logreg_cv_predict = logreg_cv.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))
print (confusion_matrix(y_test , logreg_cv_predict))
print ("Accuracy score: ", accuracy_score(y_test , logreg_cv_predict))
print("F1 score: ", f1_score(y_test , logreg_cv_predict))
print("Precision score: " , precision_score(y_test , logreg_cv_predict))
print("Recall score: " , recall_score(y_test , logreg_cv_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , logreg_cv_predict))
print (classification_report(y_test , logreg_cv_predict))
logreg_cv

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.8min finished
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Tuned Logistic Regression Parameter: {'lr__C': 1, 'lr__max_iter': 100, 'lr__solver': 'lbfgs'}
Tuned Logistic Regression Accuracy: 0.9993140741213076
[[1906207     144]
 [   1177    1258]]
Accuracy score:  0.9993079370867137
F1 score:  0.655720615063852
Precision score:  0.8972895863052782
Recall score:  0.5166324435318275
ROC-AUC score:  0.7582784532752214
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.90      0.52      0.66      2435

    accuracy                           1.00   1908786
   macro avg       0.95      0.76      0.83   1908786
weighted avg       1.00      1.00      1.00   1908786

CPU times: user 1min 21s, sys: 6.21 s, total: 1min 27s
Wall time: 2min 36s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lr',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=100,
                                                           multi_class='warn'

In [9]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('lr', LogisticRegression())
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'lr__C': [1],
              'lr__max_iter': [1000],
              'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
             }


# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

logreg_cv_predict = logreg_cv.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))
print (confusion_matrix(y_test , logreg_cv_predict))
print ("Accuracy score: ", accuracy_score(y_test , logreg_cv_predict))
print("F1 score: ", f1_score(y_test , logreg_cv_predict))
print("Precision score: " , precision_score(y_test , logreg_cv_predict))
print("Recall score: " , recall_score(y_test , logreg_cv_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , logreg_cv_predict))
print (classification_report(y_test , logreg_cv_predict))
logreg_cv

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 159.1min finished


Tuned Logistic Regression Parameter: {'lr__C': 1, 'lr__max_iter': 1000, 'lr__solver': 'newton-cg'}
Tuned Logistic Regression Accuracy: 0.9993140741213076
[[1906206     145]
 [   1177    1258]]
Accuracy score:  0.9993074131935167
F1 score:  0.655549765502866
Precision score:  0.8966500356379188
Recall score:  0.5166324435318275
ROC-AUC score:  0.7582781909940359
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.90      0.52      0.66      2435

    accuracy                           1.00   1908786
   macro avg       0.95      0.76      0.83   1908786
weighted avg       1.00      1.00      1.00   1908786

CPU times: user 1min 54s, sys: 14.5 s, total: 2min 8s
Wall time: 2h 40min 22s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lr',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=100,
                                                           multi_class='warn'

In [8]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('lr', LogisticRegression())
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'lr__C': [1000],
              'lr__max_iter': [1000],
              'lr__solver': ['newton-cg']
             }


# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
logreg_cv.fit(X_train, y_train)

logreg_cv_predict = logreg_cv.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(logreg_cv.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(logreg_cv.best_score_))
print (confusion_matrix(y_test , logreg_cv_predict))
print ("Accuracy score: ", accuracy_score(y_test , logreg_cv_predict))
print("F1 score: ", f1_score(y_test , logreg_cv_predict))
print("Precision score: " , precision_score(y_test , logreg_cv_predict))
print("Recall score: " , recall_score(y_test , logreg_cv_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , logreg_cv_predict))
print (classification_report(y_test , logreg_cv_predict))
logreg_cv

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.3min finished


Tuned Logistic Regression Parameter: {'lr__C': 1000, 'lr__max_iter': 1000, 'lr__solver': 'newton-cg'}
Tuned Logistic Regression Accuracy: 0.9993199117883603
[[1906202     149]
 [   1165    1270]]
Accuracy score:  0.999311604339093
F1 score:  0.65905552672548
Precision score:  0.8949964763918252
Recall score:  0.5215605749486653
ROC-AUC score:  0.7607412075777134
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.89      0.52      0.66      2435

    accuracy                           1.00   1908786
   macro avg       0.95      0.76      0.83   1908786
weighted avg       1.00      1.00      1.00   1908786

CPU times: user 2min 12s, sys: 16.9 s, total: 2min 29s
Wall time: 5min 47s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lr',
                                        LogisticRegression(C=1.0,
                                                           class_weight=None,
                                                           dual=False,
                                                           fit_intercept=True,
                                                           intercept_scaling=1,
                                                           l1_ratio=None,
                                                           max_iter=100,
                                                           multi_class='warn'